<center><img src='https://i.postimg.cc/TPR1n1rp/AI-Tech-PL-RGB.png' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://i.postimg.cc/Gpq2KRQz/logotypy-aitech.jpg'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
</center>

# Lab 03: Imitation Learning

In this lab, we look into the problem of learning from expert demonstrations.

- Find a policy $\pi(a | s)$ that best imitates the expert policy $\pi^*(a | s)$ in the given environment.
- It's worth noting, that we don't need access to the environment rewards.

Major Imitation Learning techniques are:

1. Behavioural Cloning,
1. Imitation Learning via Interactive Demonstrator e.g. SMILe (Ross and Bagnell, 2010) or DAgger (Ross et al., 2011),
1. Inverse Reinforcement Learning -- out of scope of this lab.

We will solve the Ant problem, shown below, examining the first two approaches.

In [2]:
#@title Mount your Google Drive

#@markdown Your work will be stored in a folder called `rl_lab_2022` by default.

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

import os
from google.colab import drive
drive.mount('/content/gdrive')

LAB_PATH = '/content/gdrive/MyDrive/rl_lab_2022/imitation_learning'
if not os.path.exists(LAB_PATH):
  %mkdir -p $LAB_PATH

MJC_PATH = '{}/mujoco'.format(LAB_PATH)
if not os.path.exists(MJC_PATH):
    %mkdir $MJC_PATH

Mounted at /content/gdrive


In [3]:
#@title Install requirements

!apt -q update 
!apt install -q -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg
!pip -q install gdown

# Installing dependencies for visualization
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install -U gym==0.19
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 

In [4]:
#@title Download MuJoCo

if not os.path.exists(f'{MJC_PATH}/mujoco210'):
    %cd $MJC_PATH
    !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
    !tar -xzf mujoco210-linux-x86_64.tar.gz
    %rm mujoco210-linux-x86_64.tar.gz

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco


In [5]:
#@title Install MuJoCo

import os

os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco210/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco210'.format(MJC_PATH)

# Installation on colab does not find *.so files in LD_LIBRARY_PATH,
# copy over manually instead.
!cp $MJC_PATH/mujoco210/bin/*.so /usr/lib/x86_64-linux-gnu/

In [6]:
#@title Clone and install mujoco-py

if not os.path.exists(f'{MJC_PATH}/mujoco-py'):
    %cd $MJC_PATH
    !git clone https://github.com/openai/mujoco-py.git

%cd $MJC_PATH/mujoco-py
!git checkout f1312cceeeebbba17e78d5d77fbffa091eed9a3a # Tested version
%pip install -e .

# Compile at the first import
os.environ['LD_LIBRARY_PATH'] += ':/usr/lib/nvidia'
os.environ['LD_LIBRARY_PATH'] += ':/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco210/bin'
import mujoco_py

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco
Cloning into 'mujoco-py'...
remote: Enumerating objects: 2183, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2183 (delta 7), reused 20 (delta 1), pack-reused 2148
Receiving objects: 100% (2183/2183), 5.70 MiB | 7.19 MiB/s, done.
Resolving deltas: 100% (1350/1350), done.
Checking out files: 100% (200/200), done.
/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py
M	.dockerignore
M	.gitignore
M	mujoco_py/tests/test_substep.py
M	scripts/gen_wrappers.py
M	vendor/Xdummy-entrypoint
Note: checking out 'f1312cceeeebbba17e78d5d77fbffa091eed9a3a'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or 

In [7]:
#@title Download the expert checkpoint

if not os.path.exists(f'{LAB_PATH}/expert_checkpoint'):
    %cd $LAB_PATH
    !gdown --id 1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
    !unzip expert_checkpoint.zip

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_checkpoint.zip
100% 382k/382k [00:00<00:00, 57.9MB/s]
Archive:  expert_checkpoint.zip
   creating: expert_checkpoint/
  inflating: __MACOSX/._expert_checkpoint  
   creating: expert_checkpoint/variables/
  inflating: __MACOSX/expert_checkpoint/._variables  
  inflating: expert_checkpoint/saved_model.pb  
   creating: expert_checkpoint/assets/
  inflating: __MACOSX/expert_checkpoint/._assets  
  inflating: expert_checkpoint/variables/variables.data-00000-of-00001  
  inflating: expert_checkpoint/variables/variables.index  


In [8]:
import glob
import time

from functools import partial

import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from base64 import b64encode
from IPython.display import HTML
from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(1024, 768))
display.start()

# Seed random generators
tf.random.set_seed(42)
np.random.seed(42)

# Helpers

def show_video(file_name):
    mp4 = open(file_name,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=480 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

def mlp(input_shape, output_size, hidden_sizes=(256, 256), hidden_activation=tf.tanh, output_activation=None, l2_weight=0.0001):
    """Creates MLP with the specified parameters."""
    model = tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    for h in hidden_sizes:
        model.add(tf.keras.layers.Dense(units=h,
                                        activation=hidden_activation,
                                        kernel_regularizer=tf.keras.regularizers.L2(l2_weight)))
    model.add(tf.keras.layers.Dense(units=output_size, activation=output_activation))

    return model

def run_policy (env, model, total_steps=10000, verbose=True):
    obs_array = np.empty([total_steps, *env.observation_space.shape])
    act_array = np.empty([total_steps, env.action_space.shape[0]])
    rew_array = np.empty([total_steps, 1])
    done_array = np.empty([total_steps, 1])

    iter_time = time.time()
    done = True
    for i in range(total_steps):
        if verbose and (i + 1) % 1000 == 0:
            steps_per_second = 1000 / (time.time() - iter_time)
            print(f'Step {i + 1}/{total_steps}, Steps per second: {steps_per_second}')
            iter_time = time.time()


        if done:
            obs = env.reset()

        act = model(tf.expand_dims(obs, axis=0))[0]
        obs_, rew, done, _ = env.step(act)
        
        obs_array[i] = obs
        act_array[i] = act
        rew_array[i] = rew
        done_array[i] = float(done)

        obs = obs_

    return obs_array, act_array, rew_array, done_array

def calculate_returns(rew, done):
    rew_cumsum = np.cumsum(rew)[:, None]
    ret_cumsum = rew_cumsum * done
    ret_cumsum_trimed = ret_cumsum[np.nonzero(ret_cumsum)]
    ret_cumsum_trimed[1:] -= ret_cumsum_trimed[:-1]
    return ret_cumsum_trimed

def evaluate_agent(env, model, verbose=False):
    _, _, rew, done = run_policy(env, model, total_steps=25000, verbose=verbose)
    rets = calculate_returns(rew, done)

    print(f'Num. episodes: {len(rets)}')
    print(f'Avg. return: {np.mean(rets)}')
    print(f'Max. return: {np.max(rets)}')
    print(f'Min. return: {np.min(rets)}')

def render_agent(env, model):
    envw = gym.wrappers.Monitor(env, "./", force=True)
    o, d = envw.reset(), False
    while not d:
        envw.render()
        o, _, d, _ = envw.step(model(tf.expand_dims(o, axis=0))[0])
    # envw.close()

    file_name = glob.glob('openaigym.video.*.mp4')[0]
    return show_video(file_name)

class Expert:
    """Streamlined Off-Policy (SOP) actor"""

    def __init__(self, ckpt_path):
        self.model = tf.keras.models.load_model(ckpt_path)

    def __call__(self, obs, exploratory=False):
        # We need to add one more dim. for this model
        mu, pi = self.model(tf.expand_dims(obs, axis=0))
        return pi[0] if exploratory else mu[0]

## 0. Ant

a three-dimensional quadrupedal robot.

- Observations are 111-dim. vectors that describe the kinematic properties of the robot,
- Actions are 8-dim. vectors which specify torques to be applied on the robot joints,
- The goal is to run forward as fast as possible and don’t fall over.

In [9]:
%cd $LAB_PATH
expert = Expert('expert_checkpoint')
env = gym.make('Ant-v2')

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning


In [ ]:
# render_agent(env, expert)

In [10]:
evaluate_agent(env, expert, verbose=True)

Step 1000/25000, Steps per second: 68.74989653021768
Step 2000/25000, Steps per second: 123.93378940459449
Step 3000/25000, Steps per second: 120.10885758667708
Step 4000/25000, Steps per second: 113.44612389398722
Step 5000/25000, Steps per second: 120.23996828004834
Step 6000/25000, Steps per second: 115.09801103702789
Step 7000/25000, Steps per second: 109.82604516822552
Step 8000/25000, Steps per second: 110.4651536407824
Step 9000/25000, Steps per second: 119.04838472218678
Step 10000/25000, Steps per second: 118.94245306842339
Step 11000/25000, Steps per second: 115.38773148673523
Step 12000/25000, Steps per second: 81.86738581386318
Step 13000/25000, Steps per second: 115.20326188923282
Step 14000/25000, Steps per second: 122.02858724151213
Step 15000/25000, Steps per second: 120.6113511325416
Step 16000/25000, Steps per second: 119.34712577663554
Step 17000/25000, Steps per second: 127.73909539256489
Step 18000/25000, Steps per second: 130.5575177018836
Step 19000/25000, Steps 

## 1. Behaviour Clonning

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.

In [11]:
# Collect the expert data
# obs, act, _, _ = run_policy(env, expert, total_steps=100000)

# Download the expert data instead of collecting it
if not os.path.exists(f'expert_greedy_data.npz'):
    !gdown --id 1Ny7xe3aZR-u2TpaoNhDG6wI4SS5N8FR8
with np.load('expert_greedy_data.npz') as data:
    obs, act, _, _ = data.values()

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Ny7xe3aZR-u2TpaoNhDG6wI4SS5N8FR8
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_greedy_data.npz
100% 96.8M/96.8M [00:00<00:00, 126MB/s]


In [12]:
# EXERCISE: Create the imitator model
imitator = mlp(111, 8)

# We will start our experiments from the same weights for the fair comparison
init_weights = imitator.get_weights()

In [23]:
# EXERCISE: Fit the model to the expert data
imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs, act, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 11s 3ms/step - loss: 0.1020
Epoch 2/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0664
Epoch 3/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0600
Epoch 4/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0565
Epoch 5/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0542
Epoch 6/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0525
Epoch 7/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0513
Epoch 8/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0504
Epoch 9/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0497
Epoch 10/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0490
Epoch 11/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0486
Epoch 12/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0481


### Exercise

Discuss the questions

1. In principle, do we need the expert policy for BC?

  > Answer: ...

1. What are the problems with BC?

  > Answer: ...

1. How can we help BC do better?

  > Answer: ...

In [14]:
# Collect the exploratory data
def exploratory(obs):
    """Adds the Gaussian noise to the expert actions."""
    act = expert(obs)
    return act + 0.29 * tf.random.normal(tf.shape(act))
# obs_expl, act_expl, rew_expl, done_expl = run_policy(env, exploratory, total_steps=100000)

# Download instead...
if not os.path.exists(f'expert_exploratory_data.npz'):
    !gdown --id 1O-xDKmH0a8i6Q5Zfmqc6qZ0Yi8H3q5ZB
with np.load('expert_exploratory_data.npz') as data:
    obs_expl, act_expl, rew_expl, done_expl = data.values()

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1O-xDKmH0a8i6Q5Zfmqc6qZ0Yi8H3q5ZB
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_exploratory_data.npz
100% 96.8M/96.8M [00:00<00:00, 129MB/s]


In [24]:
# EXERCISE: Run BC on the exploratory data

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, act_expl, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 11s 3ms/step - loss: 0.1755
Epoch 2/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1354
Epoch 3/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1277
Epoch 4/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1240
Epoch 5/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1216
Epoch 6/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1199
Epoch 7/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1186
Epoch 8/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1175
Epoch 9/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1166
Epoch 10/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1160
Epoch 11/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1155
Epoch 12/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1150


### Exercise

Answer the questions

1. Why does it better?

  > Answer: ...

1. How can we use the expert to further improve the data?

  > Hint: Noisy actions help in collecting more diverse data, but we don't want to learn the exploratory actions.

  > Answer: ...

In [18]:
# EXERCISE: Infere the expert actions on the exploratory observations
#           and run BC on it.

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, act, epochs=25)
# ANSWER END

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 6s 2ms/step - loss: 3.3549e-04
Epoch 2/25
3125/3125 [==============================] - 6s 2ms/step - loss: 4.5492e-17
Epoch 3/25
3125/3125 [==============================] - 6s 2ms/step - loss: 1.7612e-17
Epoch 4/25
3125/3125 [==============================] - 6s 2ms/step - loss: 9.5654e-18
Epoch 5/25
3125/3125 [==============================] - 6s 2ms/step - loss: 6.0728e-18
Epoch 6/25
3125/3125 [==============================] - 6s 2ms/step - loss: 4.2244e-18
Epoch 7/25
3125/3125 [==============================] - 6s 2ms/step - loss: 3.1214e-18
Epoch 8/25
3125/3125 [==============================] - 11s 3ms/step - loss: 2.4075e-18
Epoch 9/25
3125/3125 [==============================] - 6s 2ms/step - loss: 1.9176e-18
Epoch 10/25
3125/3125 [==============================] - 6s 2ms/step - loss: 1.5661e-18
Epoch 11/25
3125/3125 [==============================] - 8s 2ms/step - loss: 1.3048e-18
Epoch 12/25
3125/3125 [=================

### Exercise

Answer the questions

1. Did it help? Why?

  > Answer: ...

1. How can you extend this idea?

  > Hint: How can we get more exploratory data?

  > Answer: ...

## 2. Imitation Learning via Interactive Demostrator

[DAgger](https://www.ri.cmu.edu/pub_files/2011/4/Ross-AISTATS11-NoRegret.pdf)

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.
3. Collect the imitator data.
4. Infere the expert actions on the imitator data.
5. Fit the model to the extended dataset.
6. Repeat from 3.

In [19]:
# We will pre-train on less expert data to keep the same dataset size
obs_ = obs[:30000,:]
act_ = act[:30000,:]

In [20]:
# EXERCISE: Pretrain for 25 epochs

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_, act_, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
938/938 [==============================] - 2s 2ms/step - loss: 0.0011
Epoch 2/25
938/938 [==============================] - 2s 2ms/step - loss: 3.4259e-16
Epoch 3/25
938/938 [==============================] - 2s 2ms/step - loss: 1.3936e-16
Epoch 4/25
938/938 [==============================] - 2s 2ms/step - loss: 7.7746e-17
Epoch 5/25
938/938 [==============================] - 2s 2ms/step - loss: 5.0250e-17
Epoch 6/25
938/938 [==============================] - 2s 2ms/step - loss: 3.5418e-17
Epoch 7/25
938/938 [==============================] - 2s 2ms/step - loss: 2.6441e-17
Epoch 8/25
938/938 [==============================] - 2s 2ms/step - loss: 2.0565e-17
Epoch 9/25
938/938 [==============================] - 2s 2ms/step - loss: 1.6494e-17
Epoch 10/25
938/938 [==============================] - 2s 2ms/step - loss: 1.3551e-17
Epoch 11/25
938/938 [==============================] - 2s 2ms/step - loss: 1.1349e-17
Epoch 12/25
938/938 [==============================] - 2s 2ms/step 

In [26]:
# Exercise: Implement DAgger

for i in range(7):
    print(f'\n### Iter. {i+1} ###')

    # ANSWER
    print('\n1. Data collection')
    obs_extra, _, _, _ = run_policy(env, imitator) # Collect 10k steps
   
    obs_ = np.concatenate((obs_, obs_extra), axis=0)
    act_ = np.concatenate((act_, expert(obs_extra)), axis=0)


    print('\n2. Training')
    imitator.set_weights(init_weights)

    imitator.compile(loss=tf.keras.losses.MeanSquaredError())
    imitator.fit(obs_, act_)
    # END ANSWER

    print('\n3. Evaluation')
    evaluate_agent(env, imitator)


### Iter. 1 ###

1. Data collection
Step 1000/10000, Steps per second: 336.91421319083395
Step 2000/10000, Steps per second: 393.2251670617071
Step 3000/10000, Steps per second: 416.77276510873116
Step 4000/10000, Steps per second: 419.99600862481503
Step 5000/10000, Steps per second: 416.15612522236296
Step 6000/10000, Steps per second: 414.2380503107696
Step 7000/10000, Steps per second: 421.4582040027388
Step 8000/10000, Steps per second: 412.0550270704657
Step 9000/10000, Steps per second: 415.4982903791677
Step 10000/10000, Steps per second: 400.6675397796584

2. Training
1250/1250 [==============================] - 4s 3ms/step - loss: 0.1463

3. Evaluation
Num. episodes: 31
Avg. return: 604.514207053166
Max. return: 1976.4582804542106
Min. return: -231.32535512689356

### Iter. 2 ###

1. Data collection
Step 1000/10000, Steps per second: 390.2707002013368
Step 2000/10000, Steps per second: 389.24672946009247
Step 3000/10000, Steps per second: 399.74732158208406
Step 4000/10000, 

### Note

Training the expert with the SOP algorithm (Wang et al., 2020) took 3M data samples (env. interactions). Here, we nearly match it with only 100k samples! Training from the expert can be much more efficient than reinforcement learning.